# Exploration
This notebook is divided into two parts, **exploration** and **analysis** in which we clean and analyse the data indepth inorder to give a clear recomendation

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sqlite3
conn = sqlite3.connect('data/im.db')

pd.options.display.float_format = '{:,.0f}'.format

# 1. Data Exploration and Cleaning

These three are per the datasets descibed in analysis.ipymb and note that we use sqlite to open im.db as it is a sqlite3 db

In [ ]:
# get data
imdb_basics = pd.read_sql_query('SELECT * FROM movie_basics', conn)
imdb_ratings = pd.read_sql_query('SELECT * FROM movie_ratings', conn)
bom_movies_gross = pd.read_csv('data/bom.movie_gross.csv')

To get a better sense of what is in this dataset, we will have to go back to analysis.ipynb under the business problem, and we see that this is a company that has no idea of what this industry is about. With that, it's safe to say we need data that is full, which describes each film well, and that is the whole point of this section. We look at each dataframe individually and ask ourselves what this dataset has to offer that the others don't. So, what I have opted to do is to take pieces from each dataset, then combine them into one complete, somewhat clean dataframe that will give us a better sense of what we are to work with.

## 1. Cleaning 'imdb_basics'

In [ ]:
imdb_basics.info()

In [ ]:
imdb_basics.head()

---
From this we can already see that 'primary_title' and 'original_title' differ, and with that we will opt to use 'primary_title' from here on out. To get a better sense of what I am saying, you can look at the cell below.

---

In [ ]:
imdb_basics[imdb_basics['primary_title'] != imdb_basics['original_title']]

In [ ]:
# drop 'original_title'

imdb_basics = imdb_basics.drop('original_title', axis=1)

In [ ]:

imdb_basics.head()

---
filling in missing values using the median inorder to maintain centrality of the data

---

In [ ]:
median_runtime = imdb_basics['runtime_minutes'].median()
imdb_basics['runtime_minutes'] = imdb_basics['runtime_minutes'].fillna(median_runtime)
imdb_basics['genres'] = imdb_basics['genres'].fillna('Unknown')
imdb_basics.info()

## 2. Cleaning 'imdb_ratings'

In [ ]:
# ratings
imdb_ratings.info()

In [ ]:
imdb_ratings.head()

---
From the cells above you can see no cleaning was nessesary so no action was taken.

---

## 3. Cleaning 'bom_movies_gross'

In [ ]:
# movies gross
bom_movies_gross.info()

In [ ]:
bom_movies_gross.head()

---
filling empty 'studio' values with 'Unknown'.

i opted to use `0` in 'foreign_gross' rather than median as a large number of the column was empty and filling it with the median would result in inaccurate analysis.

i did however use median in 'domestic_gross' as less than `1%` of the data was missing.

---

In [ ]:
# clean movies_gross
bom_movies_gross['studio'] = bom_movies_gross['studio'].fillna('Unknown')

# convert foreign gross to float
bom_movies_gross['foreign_gross'] = bom_movies_gross['foreign_gross'].fillna(0)
bom_movies_gross['foreign_gross'] = pd.to_numeric(bom_movies_gross['foreign_gross'], errors='coerce')

median_domestic = bom_movies_gross['domestic_gross'].median()
bom_movies_gross['domestic_gross'] = bom_movies_gross['domestic_gross'].fillna(median_domestic)

bom_movies_gross.info()

## 4. Combining the cleaned dataframes

Now we can combine the best of three to start to analyse

In [ ]:
joined_imdb = pd.merge(imdb_basics, imdb_ratings, on='movie_id', how='inner')

In [ ]:
joined_imdb.info()

In [ ]:
movies = pd.merge(joined_imdb, bom_movies_gross, left_on='primary_title', right_on='title', how='inner')

In [ ]:
movies.info()

# 2. Analysis

Now that our data cleaned and organized, we can now analyse our data based on gross value and popularity:
1. Most successful studios
2. Most successful genres 
3. Trends overtime
4. Runtime of succesful films

In [ ]:
movies.info()

## 1. Most succesful studios

Why most successful studios? I presume that Microsoft would want to know who the big players in the industry are in order to know what they are up against and for them to use the data analysed as a benchmark of what is to be expected of a good studio.

In [ ]:
studio_performance = movies.groupby('studio').agg(
    total_films=('title', 'count'),
    avg_domestic_gross=('domestic_gross', 'mean'),
    avg_foreign_gross=('foreign_gross', 'mean'),
    total_domestic_gross=('domestic_gross', 'sum'),
    total_foreign_gross=('foreign_gross', 'sum')
).sort_values(by='total_domestic_gross', ascending=False)

In [ ]:
top_studios = studio_performance.head(20)
top_studios

In [ ]:
studio_names = top_studios.index
film_numbers = top_studios.total_films
total_domestic_gross = top_studios.total_domestic_gross
total_foreign_gross = top_studios.total_foreign_gross
avg_domestic_gross = top_studios.avg_domestic_gross
avg_foreign_gross = top_studios.avg_foreign_gross

fig, ((domestic_total, foreign_total), (domestic_avg, foreign_avg)) = plt.subplots(figsize=(12, 10), nrows=2, ncols=2)

# Set a title for the entire figure
fig.suptitle('Analysis of Film Studio Performance', fontsize=16)

# Plot for Total Domestic Gross vs. Number of Films
domestic_total.scatter(x=film_numbers, y=total_domestic_gross)
domestic_total.set_title('Total Domestic Gross vs. Number of Films')
domestic_total.set_xlabel('Number of Films')
domestic_total.set_ylabel('Total Domestic Gross')
domestic_total.grid(True)

# Plot for Total Foreign Gross vs. Number of Films
foreign_total.scatter(x=film_numbers, y=total_foreign_gross)
foreign_total.set_title('Total Foreign Gross vs. Number of Films')
foreign_total.set_xlabel('Number of Films')
foreign_total.set_ylabel('Total Foreign Gross')
foreign_total.grid(True)

# Plot for Average Domestic Gross vs. Number of Films
domestic_avg.scatter(x=film_numbers, y=avg_domestic_gross)
domestic_avg.set_title('Average Domestic Gross vs. Number of Films')
domestic_avg.set_xlabel('Number of Films')
domestic_avg.set_ylabel('Average Domestic Gross')
domestic_avg.grid(True)

# Plot for Average Foreign Gross vs. Number of Films
foreign_avg.scatter(x=film_numbers, y=avg_foreign_gross)
foreign_avg.set_title('Average Foreign Gross vs. Number of Films')
foreign_avg.set_xlabel('Number of Films')
foreign_avg.set_ylabel('Average Foreign Gross')
foreign_avg.grid(True)

# Improve layout to prevent titles from overlapping
plt.tight_layout(rect=[0, 0, 1, 0.96]) 

In [ ]:
# corelations:
correlations = top_studios.corr()
corr_total_domestic = correlations.loc['total_films', 'total_domestic_gross']
corr_total_foreign = correlations.loc['total_films', 'total_foreign_gross']
corr_avg_domestic = correlations.loc['total_films', 'avg_domestic_gross']
corr_avg_foreign = correlations.loc['total_films', 'avg_foreign_gross']

print(f"Correlation between Total Films and Total Domestic Gross: {corr_total_domestic:.2f}")
print(f"Correlation between Total Films and Total Foreign Gross: {corr_total_foreign:.2f}")
print(f"Correlation between Total Films and Average Domestic Gross: {corr_avg_domestic:.2f}")
print(f"Correlation between Total Films and Average Foreign Gross: {corr_avg_foreign:.2f}")

---
with this we know now the correlation between number of films produced and gross 

---

## 2. Most successful genres

Goal of this section is to identify now which of the many genres out there Microsoft should start with. 

In [ ]:
movies['total_gross'] = movies['domestic_gross'] + movies['foreign_gross']

film_genres = movies.groupby('genres').agg(
    avg_ratings=('averagerating', 'mean'),
    avg_votes=('numvotes', 'mean'),
    total_gross=('total_gross', 'sum')
).sort_values(by='avg_votes', ascending=False)
top_genres = film_genres.head(20)

In [ ]:
top_genres

In [ ]:
fig, (ratings, votes, gross) = plt.subplots(figsize=(14, 12), ncols=3)

ratings.bar(x=top_genres.index, height=top_genres['avg_ratings'])

ratings.set_title('Average Ratings by Genre', fontsize=14)
ratings.set_ylabel('Average Rating')
ratings.set_xlabel('Genre')

ratings.set_xticks(range(len(top_genres.index)))
ratings.set_xticklabels(top_genres.index, rotation=90, ha='right')
ratings.grid(axis='y')

votes.bar(x=top_genres.index, height=top_genres['avg_votes'])

votes.set_title('Average Votes by Genre', fontsize=14)
votes.set_ylabel('Average Votes')
votes.set_xlabel('Genre')

votes.set_xticks(range(len(top_genres.index)))
votes.set_xticklabels(top_genres.index, rotation=90, ha='right')
votes.grid(axis='y')

gross.bar(x=top_genres.index, height=top_genres['total_gross'])

gross.set_title('Total Gross by Genre', fontsize=14)
gross.set_ylabel('Total Gross')
gross.set_xlabel('Genre')

gross.set_xticks(range(len(top_genres.index)))
gross.set_xticklabels(top_genres.index, rotation=90, ha='right')
gross.grid(axis='y')

fig.suptitle('Analysis of Most Popular Genres', fontsize=16);

---
Now with this chart they know where to start from and just how much they are expected to get in return

---

## 3. Trends over time

Goal of this section is to know what is relevant in the current market of film

In [ ]:
genre_trends = movies.groupby(['start_year', 'genres'])['total_gross'].sum()

top_5_genres = movies.groupby('genres')['total_gross'].sum().nlargest(5).index
# Filter the genre_trends Series to include only the top 5 genres
filtered_trends = genre_trends.loc[:, top_5_genres]

# Unstack the data to pivot the genres into columns for plotting
plot_data = filtered_trends.unstack(level='genres').fillna(0)

# Create the line plot
plt.figure(figsize=(15, 8))
for genre in top_5_genres:
    plt.plot(plot_data.index, plot_data[genre], marker='o', label=genre)

plt.title('Total Gross of Top 5 Genre Combinations Over the Years', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Gross (in billions)', fontsize=12)
plt.xticks(plot_data.index)
plt.legend(title='Genre Combination')
plt.tight_layout()

---
with this it is evident that they should focus more on adventure, action, animation and comedy side

---

## 4. Runtime of succesful films

Goal is to know how long a typical succesful film should be (in minutes)

In [ ]:
runtime_success = movies.groupby(['runtime_minutes', 'averagerating']).agg(
    total_gross=('total_gross', 'sum')
).sort_values(by='total_gross', ascending=False)

In [ ]:
# Get top 50 profitable combinations
top_50 = runtime_success.head(50)

# Extract values
runtimes = top_50.index.get_level_values('runtime_minutes')
ratings = top_50.index.get_level_values('averagerating')
grosses = top_50['total_gross'] / 1e6  

plt.figure(figsize=(12, 7))
scatter = plt.scatter(runtimes, ratings, s=grosses, alpha=0.6, c=grosses, cmap='viridis', edgecolor='black')

plt.colorbar(scatter, label='Total Gross ($M)')
plt.xlabel('Runtime (minutes)')
plt.ylabel('Average IMDb Rating')
plt.title('Total Gross by Movie Runtime and Rating')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
print("Median Rating:", np.median(ratings))
print("Median Runtime:", np.median(runtimes))
print("Median Gross:", np.median(grosses) * 1e6, "")